In [1]:
import pandas as pd
import numpy as np

e = 1e1

def sigmoid(x, grad = True):
    value = 1/(1 + np.exp(-1*x)) 
    if grad:
        grad = value*(1-value)
        return value, grad 
    return value

class Perceptron:
    def __init__(self, input_shape, output_shape, activation = sigmoid):
        self.weight = np.random.normal(size=(input_shape, output_shape))
        self.activation = activation
    
    def __call__(self, inp, grad):
        output = self.weight.T @ inp 
        if grad:
            output, grad = self.activation(output, grad)
            return output, grad
        output = self.activation(output, grad)
        return output


class NeuralNet:
    def __init__(self, input_shape, output_shape, num_hidden_layers, num_hidden_neurons):
        assert num_hidden_layers > 0, "There should be at least one hidden layer"
        if type(num_hidden_neurons) != list:
            num_hidden_neurons = [num_hidden_neurons] * num_hidden_layers
        assert num_hidden_layers == len(num_hidden_neurons), "The Number of Hidden Neurons should be in Number of Hidden Layers"
        self.layers = [Perceptron(input_shape, num_hidden_neurons[0])]
        for i in range(num_hidden_layers-1):
            self.layers.append(Perceptron(num_hidden_neurons[i], num_hidden_neurons[i+1]))
        self.layers.append(Perceptron(num_hidden_neurons[-1], output_shape))
        self.grad = True
    
    def __call__(self, input):
        self.inputs = [input]
        self.gradients = []
        output, grad = self.layers[0](input, self.grad)
        self.gradients.append(grad)
        for layer in self.layers[1:]:
            self.inputs.append(output)
            output, grad = layer(output, self.grad)
            self.gradients.append(grad)
        return output
    
    def update_weights(self, grad_loss):
        grad = self.gradients[-1] * grad_loss
        self.layers[-1].weight -= grad * np.expand_dims(self.inputs[-1], axis=-1)
        for i in range(len(self.layers)-1)[::-1]:
            grad = np.expand_dims(self.gradients[i], axis=-1) * self.layers[i+1].weight @ grad            
            self.layers[i].weight -= self.lr * np.expand_dims(self.inputs[i], axis=-1) @ np.expand_dims(grad, axis=-1).T 

    def train(self, X_train, y_train, X_test, y_test, learning_rate, n_epochs, logging_epochs = 10, validation_epochs = 100):
        validation_epochs = min(validation_epochs, n_epochs)
        self.lr = learning_rate
        for current_epoch in range(0, n_epochs + 1):
            total_loss = 0
            train_accuracy = 0
            for x, y in zip(X_train, y_train):
                pred = self(x)
                loss, grad_loss = MSELoss(pred, y)
                total_loss += loss
                train_accuracy += (y == (pred>=0.5))
                self.update_weights(grad_loss)
            if current_epoch % logging_epochs == 0:
                print("\nEpoch: ", current_epoch, "\nLoss: ", total_loss/len(X_train), "\nTrain Accuracy: ", train_accuracy/len(X_train))
            if current_epoch % validation_epochs == 0:
                val_loss = 0
                val_accuracy = 0
                for x, y in zip(X_test, y_test):
                    pred = self(x)
                    loss, _ = MSELoss(pred, y)
                    val_loss += loss/len(X_test)
                    val_accuracy += (y == (pred>=0.5))
                print("Validation Loss: ", total_loss/len(X_test), "\nValidation Accuracy: ", val_accuracy/len(X_test))
        print("\nTraining Finished!")
        print("Epoch: ", current_epoch, "\nLoss: ", total_loss/len(X_train), "\nTrain Accuracy: ", train_accuracy/len(X_train))
        val_loss = 0
        val_accuracy = 0
        for x, y in zip(X_test, y_test):
            pred = self(x)
            loss, _ = MSELoss(pred, y)
            val_loss += loss/len(X_test)
            val_accuracy += (y == (pred>=0.5))
        print("Validation Loss: ", total_loss/len(X_test), "\nValidation Accuracy: ", val_accuracy/len(X_test))

def MSELoss(prediction, truth):
    loss = 0.5 * (prediction - truth) ** 2
    grad = prediction - truth
    return loss, grad

# Q1

In [16]:
X_train = pd.read_csv("trainData.csv", header=None).to_numpy()
y_train = pd.read_csv("trainLabels.csv", header=None).to_numpy()
X_test = pd.read_csv("testData.csv", header=None).to_numpy()
y_test = pd.read_csv("testLabels.csv", header=None).to_numpy()
nn = NeuralNet(64, 1, 1, 15)
nn.train(X_train, y_train - 5, X_test, y_test - 5, 0.01, 1000, 300, 300)


Epoch:  0 
Loss:  [0.10456875] 
Train Accuracy:  [0.6998672]
Validation Loss:  [0.21691533] 
Validation Accuracy:  [0.71349862]

Epoch:  300 
Loss:  [0.01834755] 
Train Accuracy:  [0.96148738]
Validation Loss:  [0.0380598] 
Validation Accuracy:  [0.8815427]

Epoch:  600 
Loss:  [0.01837459] 
Train Accuracy:  [0.96015936]
Validation Loss:  [0.03811589] 
Validation Accuracy:  [0.8815427]

Epoch:  900 
Loss:  [0.01842293] 
Train Accuracy:  [0.96015936]
Validation Loss:  [0.03821616] 
Validation Accuracy:  [0.88429752]

Training Finished!
Epoch:  1000 
Loss:  [0.01779644] 
Train Accuracy:  [0.96281541]
Validation Loss:  [0.03691659] 
Validation Accuracy:  [0.88429752]


In [17]:
nn = NeuralNet(64, 1, 1, 5)
nn.train(X_train, y_train - 5, X_test, y_test - 5, 0.01, 1000, 300, 300)


Epoch:  0 
Loss:  [0.13721985] 
Train Accuracy:  [0.52855246]
Validation Loss:  [0.28464613] 
Validation Accuracy:  [0.50413223]

Epoch:  300 
Loss:  [0.02247391] 
Train Accuracy:  [0.94555113]
Validation Loss:  [0.04661943] 
Validation Accuracy:  [0.90358127]

Epoch:  600 
Loss:  [0.01824408] 
Train Accuracy:  [0.95484728]
Validation Loss:  [0.03784516] 
Validation Accuracy:  [0.92011019]

Epoch:  900 
Loss:  [0.01721666] 
Train Accuracy:  [0.95484728]
Validation Loss:  [0.0357139] 
Validation Accuracy:  [0.92011019]

Training Finished!
Epoch:  1000 
Loss:  [0.01376572] 
Train Accuracy:  [0.96148738]
Validation Loss:  [0.02855534] 
Validation Accuracy:  [0.93112948]


In [18]:
nn = NeuralNet(64, 1, 1, 5)
nn.train(X_train, y_train - 5, X_test, y_test - 5, 0.001, 1000, 300, 300)


Epoch:  0 
Loss:  [0.12766984] 
Train Accuracy:  [0.53652058]
Validation Loss:  [0.26483579] 
Validation Accuracy:  [0.49311295]

Epoch:  300 
Loss:  [0.086998] 
Train Accuracy:  [0.74236388]
Validation Loss:  [0.18046692] 
Validation Accuracy:  [0.69421488]

Epoch:  600 
Loss:  [0.05321668] 
Train Accuracy:  [0.85258964]
Validation Loss:  [0.11039163] 
Validation Accuracy:  [0.79889807]

Epoch:  900 
Loss:  [0.04466667] 
Train Accuracy:  [0.88579017]
Validation Loss:  [0.09265565] 
Validation Accuracy:  [0.84297521]

Training Finished!
Epoch:  1000 
Loss:  [0.04367172] 
Train Accuracy:  [0.88446215]
Validation Loss:  [0.09059175] 
Validation Accuracy:  [0.84848485]


In [19]:
nn = NeuralNet(64, 1, 1, 15)
nn.train(X_train, y_train - 5, X_test, y_test - 5, 0.001, 1000, 300, 300)


Epoch:  0 
Loss:  [0.13153826] 
Train Accuracy:  [0.63081009]
Validation Loss:  [0.27286036] 
Validation Accuracy:  [0.75757576]

Epoch:  300 
Loss:  [0.05511808] 
Train Accuracy:  [0.87383798]
Validation Loss:  [0.11433585] 
Validation Accuracy:  [0.83746556]

Epoch:  600 
Loss:  [0.0384547] 
Train Accuracy:  [0.92297477]
Validation Loss:  [0.07976967] 
Validation Accuracy:  [0.83471074]

Epoch:  900 
Loss:  [0.03241347] 
Train Accuracy:  [0.93359894]
Validation Loss:  [0.06723786] 
Validation Accuracy:  [0.84022039]

Training Finished!
Epoch:  1000 
Loss:  [0.03116379] 
Train Accuracy:  [0.93625498]
Validation Loss:  [0.06464555] 
Validation Accuracy:  [0.83746556]


# Q2

In [3]:
data = pd.read_csv("iris.csv")
data.columns

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

In [4]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [5]:
data.variety.unique()

array(['Setosa', 'Versicolor'], dtype=object)

In [6]:
def convert(x):
    values = ["Setosa", "Versicolor"]
    return values.index(x)

data["variety"] = data["variety"].apply(convert)

In [7]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
len(data)

100

In [9]:
# Shuffle the Dataset
data = data.sample(frac = 1)

In [10]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
63,6.1,2.9,4.7,1.4,1
11,4.8,3.4,1.6,0.2,0
62,6.0,2.2,4.0,1.0,1
23,5.1,3.3,1.7,0.5,0
75,6.6,3.0,4.4,1.4,1


In [11]:
np_data = data.to_numpy()

In [12]:
size = len(np_data)
fraction = 0.8
train_data = np_data[:int(fraction*size)]
test_data = np_data[int(fraction*size):]

In [13]:
X_train, y_train = train_data[:,:-1], train_data[:,-1]
X_test, y_test = test_data[:,:-1], test_data[:,-1]

In [14]:
np_data.shape, train_data.shape, test_data.shape

((100, 5), (80, 5), (20, 5))

In [15]:
nn = NeuralNet(4, 1, num_hidden_layers = 3, num_hidden_neurons = [4,8,4])
nn.train(X_train, y_train, X_test, y_test, 0.001, 1000, 300, 300)


Epoch:  0 
Loss:  [0.11757197] 
Train Accuracy:  [0.5625]
Validation Loss:  [0.47028786] 
Validation Accuracy:  [0.45]

Epoch:  300 
Loss:  [0.00055898] 
Train Accuracy:  [1.]
Validation Loss:  [0.00223592] 
Validation Accuracy:  [1.]

Epoch:  600 
Loss:  [0.0002325] 
Train Accuracy:  [1.]
Validation Loss:  [0.00092999] 
Validation Accuracy:  [1.]

Epoch:  900 
Loss:  [0.00014268] 
Train Accuracy:  [1.]
Validation Loss:  [0.0005707] 
Validation Accuracy:  [1.]

Training Finished!
Epoch:  1000 
Loss:  [0.00012597] 
Train Accuracy:  [1.]
Validation Loss:  [0.00050389] 
Validation Accuracy:  [1.]
